In [ ]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

from torch.nn.parameter import Parameter
import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat
#Set default dtype to float32
torch.set_default_dtype(torch.float)

from smt.sampling_methods import LHS
#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

In [ ]:
data = scipy.io.loadmat('burgers_shock_10sin.mat')
label = "QCRE_2D_5_tanhxtanh_NW"
                     
x_test = data['x']                                   # 256 points between -1 and 1 [256x1]
t_test = data['t']   
usol = data['usol']
X_test, T_test = np.meshgrid(x_test,t_test)  

xt_test_tensor = torch.from_numpy(np.hstack((X_test.flatten()[:,None], T_test.flatten()[:,None]))).float().to(device)

u_true = usol.flatten('F')[:,None]

In [3]:
# data = scipy.io.loadmat('burgers_shock_10sin.mat')  	# Load data from file

pi = torch.from_numpy(np.array(np.pi)).double().to(device)

label = "QCRE_2D_5_tanhxtanh_NW"
# x = data['x']                                   # 256 points between -1 and 1 [256x1]
# t = data['t']                                   # 100 time points between 0 and 0.2 [100x1] 
# usol = data['usol']   

#usol = usol/1000# solution of 256x100 grid points

x = np.linspace(-1,1,500).reshape(-1,1)
t = np.linspace(0,0.2,500).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

#y_true = true_2D_1(xt)

bound_pts_1 = (X==-1).reshape(-1,)
xt_bound_1 = xt[bound_pts_1,:]
u_bound_1 = np.zeros((np.shape(xt_bound_1)[0],1))

bound_pts_2 = (X==1).reshape(-1,)
xt_bound_2 = xt[bound_pts_2,:]
u_bound_2 = np.zeros((np.shape(xt_bound_2)[0],1))

bound_pts_3 = (T==0).reshape(-1,)
xt_bound_3 = xt[bound_pts_3,:]
u_bound_3 = -10*np.sin(np.pi*xt_bound_3[:,0].reshape(-1,1))
#u_bound_3 = -10*np.ones((np.shape(bound_pts_3)[0],1))


xt_bound = np.vstack((xt_bound_1,xt_bound_2,xt_bound_3))
u_bound = np.vstack((u_bound_1,u_bound_2,u_bound_3))


lb_xt = xt[0]
ub_xt = xt[-1]

In [ ]:
plt.plot(xt_bound_3[:,0],u_bound_3)

In [11]:
def pretrainingdata(N_u1,N_u2,seed):

    '''Boundary Conditions'''
    #Initial Condition -1 =< x =<1 and t = 0  
#     leftedge_x = np.hstack((X[0,:][:,None], T[0,:][:,None])) #L1
#     leftedge_u = usol[:,0][:,None]

#     #Boundary Condition x = -1 and 0 =< t =<1
#     bottomedge_x = np.hstack((X[:,0][:,None], T[:,0][:,None])) #L2
#     bottomedge_u = usol[-1,:][:,None]

#     #Boundary Condition x = 1 and 0 =< t =<1
#     topedge_x = np.hstack((X[:,-1][:,None], T[:,0][:,None])) #L3
#     topedge_u = usol[0,:][:,None]

#     all_X_u_train = np.vstack([leftedge_x, bottomedge_x, topedge_x]) # X_u_train [456,2] (456 = 256(L1)+100(L2)+100(L3))
#     all_u_train = np.vstack([leftedge_u, bottomedge_u, topedge_u])   #corresponding u [456x1]

    #choose random N_u points for training
    np.random.seed(seed)
    idx = np.random.choice(xt_bound.shape[0], N_u1, replace=True) 

    X_u_train = xt_bound[idx, :] #choose indices from  set 'idx' (x,t)
    u_train = u_bound[idx,:]      #choose corresponding u

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_u2)
    
    X_u2_train = lb_xt + (ub_xt-lb_xt)*samples 
    
    abc = (X_u2_train[:,1]*10 + (X_u2_train[:,0] - 1) + (X_u2_train[:,1] + 1))
    a1 = X_u2_train[:,1]*10/abc
#     a2 = (X_u2_train[:,0] - 1)/abc
#     a3 = 1-a2-a3
    
    
    
    u2_train = -10*a1.reshape(-1,1)

    X_u_train = np.vstack((X_u_train, X_u2_train))
    u_train = np.vstack((u_train, u2_train))
    
    return X_u_train, u_train

In [5]:
def trainingdata(N_u,N_f,seed):

    '''Boundary Conditions'''
    #Initial Condition -1 =< x =<1 and t = 0  
#     leftedge_x = np.hstack((X[0,:][:,None], T[0,:][:,None])) #L1
#     leftedge_u = usol[:,0][:,None]

#     #Boundary Condition x = -1 and 0 =< t =<1
#     bottomedge_x = np.hstack((X[:,0][:,None], T[:,0][:,None])) #L2
#     bottomedge_u = usol[-1,:][:,None]

#     #Boundary Condition x = 1 and 0 =< t =<1
#     topedge_x = np.hstack((X[:,-1][:,None], T[:,0][:,None])) #L3
#     topedge_u = usol[0,:][:,None]

#     all_X_u_train = np.vstack([leftedge_x, bottomedge_x, topedge_x]) # X_u_train [456,2] (456 = 256(L1)+100(L2)+100(L3))
#     all_u_train = np.vstack([leftedge_u, bottomedge_u, topedge_u])   #corresponding u [456x1]

    #choose random N_u points for training
    np.random.seed(seed)
    idx = np.random.choice(xt_bound.shape[0], N_u, replace=True) 

    X_u_train = xt_bound[idx, :] #choose indices from  set 'idx' (x,t)
    u_train = u_bound[idx,:]      #choose corresponding u

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    X_f_train = lb_xt + (ub_xt-lb_xt)*samples 
    
    X_f_train = np.vstack((X_f_train, X_u_train)) # append training points to collocation points 

    return X_f_train, X_u_train, u_train

In [6]:
#X_u_test = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
X_u_test = xt_test_tensor
# Domain bounds
lb = X_u_test[0]  # [-1. 0.]
ub = X_u_test[-1] # [1.  0.99]

'''
   Fortran Style ('F') flatten,stacked column wise!
   u = [c1 
        c2
        .
        .
        cn]

   u =  [25600x1] 
'''
#u_true = usol.flatten('F')[:,None]

"\n   Fortran Style ('F') flatten,stacked column wise!\n   u = [c1 \n        c2\n        .\n        .\n        cn]\n\n   u =  [25600x1] \n"

In [12]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        '''
        Alternatively:
        
        *all layers are callable 
    
        Simple linear Layers
        self.fc1 = nn.Linear(2,50)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,50)
        self.fc4 = nn.Linear(50,1)
        
        '''
    
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        self.beta = Parameter(1*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.train_loss = []
        self.test_loss = []
        self.beta_val = []
        
        self.W1 = Parameter(torch.tensor(0.0))
        self.W1.requiresGrad = True
        
        self.W2 = Parameter(torch.tensor(0.0))
        self.W2.requiresGrad = True
        
    'foward pass'
    def forward(self,x):
         if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
         u_b = torch.from_numpy(ub_xt).float().to(device)
         l_b = torch.from_numpy(lb_xt).float().to(device)
        
            
         #preprocessing input 
         x = (x - l_b)/(u_b - l_b) #feature scaling
        
         #convert to float
         a = x.float()
                        
         '''     
         Alternatively:
        
         a = self.activation(self.fc1(a))
         a = self.activation(self.fc2(a))
         a = self.activation(self.fc3(a))
         a = self.fc4(a)
         
         '''
        
         for i in range(len(layers)-2):
                z = self.linears[i](a)
                a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
         
         a = self.linears[-1](a)
        
         return a
                        
    def loss_BC(self,x,y):
                
        loss_u = self.loss_function(self.forward(x), y)
                
        return loss_u
    
    def loss_PDE(self, x_to_train_f,f_hat):
        
        nu = 0.01/pi
                
        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
                        
        g = x_to_train_f.clone()
                        
        g.requires_grad = True
        
        u = self.forward(g)
                
        u_x_t = autograd.grad(u,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
                                
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(x_to_train_f.shape).to(device), create_graph=True)[0]
                                                            
        u_x = u_x_t[:,[0]]
        
        u_t = u_x_t[:,[1]]
        
        u_xx = u_xx_tt[:,[0]]
                                        
        f = u_t + (self.forward(g))*(u_x) - (nu)*u_xx 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,x,y,x_to_train_f,f_hat):

        loss_u = self.loss_BC(x,y)
        loss_f = self.loss_PDE(x_to_train_f,f_hat)
        
        p = torch.exp(self.W1)
        q = torch.exp(self.W2)
        
        
        loss_val = (1+p)*loss_u + (1+q)*loss_f
        
        return loss_val
    
#     def closure_pretrain(self):
#         optimizer.zero_grad()        

#         loss = self.loss_BC(X_u_train, u_train)
        
#         self.train_loss.append(loss.cpu().detach().numpy())
        
#         u_pred = self.test(xt_test_tensor)
#         self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1))))
#         self.beta_val.append(self.beta.cpu().detach().numpy())
        
#         print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
# #         print(self.iter,"Train Loss",self.train_loss[-1])
#         loss.backward()
#         self.iter += 1
     
#         return loss  
        
     
    'callable for optimizer'                                       
    def closure(self):
        optimizer.zero_grad()
        
        if (flag==1):
            loss = self.loss(X_u_train, u_train, X_f_train,f_hat)
        else:
            loss = self.loss_BC(X_u_train, u_train)
            
        self.train_loss.append(loss.cpu().detach().numpy())


        u_pred = self.test(xt_test_tensor)
        self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1))))
        self.beta_val.append(self.beta.cpu().detach().numpy())

        print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
    #   print(self.iter,"Train Loss",self.train_loss[-1])
        loss.backward()
        self.iter += 1
            
            

            

        
     
        return loss        
    'test neural network'
    
    def test(self,xt_test_tensor):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

In [13]:
##PRE-TRAIN
max_reps = 1
flag = 0

train_loss_full = []
test_loss_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))


for reps in range(max_reps):
    print(reps)
    N_u1 = 1000 #Total number of data points for 'u'
    N_u2 = 10000
    
    X_u_train, u_train = pretrainingdata(N_u1,N_u2,reps*23)
        
    X_u_train = torch.from_numpy(X_u_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)


    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)
       
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    'L-BFGS Optimizer'
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                                  max_iter = 10000, 
                                  max_eval = None, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')

    start_time = time.time()

    optimizer.step(PINN.closure)
    

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0 Train Loss 27717.355 Test Loss 24.60512888128238
1 Train Loss 27716.02 Test Loss 24.552168426891523
2 Train Loss 27706.166 Test Loss 24.599322477254148
3 Train Loss 27682.96 Test Loss 29.428634868979106
4 Train Loss 27649.652 Test Loss 58.596709229092895
5 Train Loss 27693.908 Test Loss 274.454174

154 Train Loss 23892.707 Test Loss 2450.0361523174665
155 Train Loss 23896.395 Test Loss 2586.1574923157527
156 Train Loss 23848.988 Test Loss 2513.136106766921
157 Train Loss 23805.473 Test Loss 2679.669038407352
158 Train Loss 23756.162 Test Loss 3002.7247375719244
159 Train Loss 23748.725 Test Loss 3007.2870677722512
160 Train Loss 23720.734 Test Loss 2985.865613894776
161 Train Loss 24489.992 Test Loss 2578.454741601559
162 Train Loss 23676.09 Test Loss 2903.43290719367
163 Train Loss 23832.33 Test Loss 2608.782523197115
164 Train Loss 23630.705 Test Loss 2818.1199104429134
165 Train Loss 23747.352 Test Loss 3006.051919615291
166 Train Loss 23597.67 Test Loss 2858.7080623164657
167 Train Loss 23502.818 Test Loss 3174.216715881537
168 Train Loss 23409.58 Test Loss 3595.032833307661
169 Train Loss 23391.1 Test Loss 3460.3725064062432
170 Train Loss 23368.23 Test Loss 3654.63974591603
171 Train Loss 23226.236 Test Loss 3167.8281159730436
172 Train Loss 23413.771 Test Loss 3005.2047374

318 Train Loss 20559.053 Test Loss 6080.41879836186
319 Train Loss 20556.113 Test Loss 6147.096743351958
320 Train Loss 20552.918 Test Loss 6079.998976496735
321 Train Loss 20551.19 Test Loss 6080.783719036629
322 Train Loss 20548.408 Test Loss 6085.878500892943
323 Train Loss 20544.467 Test Loss 6102.7028855150575
324 Train Loss 20535.744 Test Loss 6141.102253156405
325 Train Loss 20531.521 Test Loss 6154.335398107003
326 Train Loss 20525.951 Test Loss 6146.256875354361
327 Train Loss 20523.459 Test Loss 6164.2643958024555
328 Train Loss 20519.818 Test Loss 6198.189175652533
329 Train Loss 20514.5 Test Loss 6190.073559949782
330 Train Loss 20503.75 Test Loss 6185.795815425405
331 Train Loss 17904060.0 Test Loss 18360726.88019323
332 Train Loss 20511.318 Test Loss 6191.9441709432285
333 Train Loss 20502.887 Test Loss 6184.69141924617
334 Train Loss 20499.557 Test Loss 6211.661475877222
335 Train Loss 20485.268 Test Loss 6120.667897616861
336 Train Loss 20485.768 Test Loss 5994.20132742

482 Train Loss 20038.941 Test Loss 6953.11545949383
483 Train Loss 19943.264 Test Loss 6735.4247528272845
484 Train Loss 19941.81 Test Loss 6750.616516020612
485 Train Loss 20034.855 Test Loss 7005.516467992015
486 Train Loss 19940.02 Test Loss 6769.606280315913
487 Train Loss 19934.838 Test Loss 6754.14222304089
488 Train Loss 19930.71 Test Loss 6658.063084576749
489 Train Loss 19925.098 Test Loss 6693.031318841346
490 Train Loss 19924.197 Test Loss 6805.196724627703
491 Train Loss 19922.021 Test Loss 6754.362875997323
492 Train Loss 19920.738 Test Loss 6816.38171608892
493 Train Loss 19918.754 Test Loss 6818.536005204823
494 Train Loss 19915.168 Test Loss 6804.7668999753705
495 Train Loss 32150.355 Test Loss 20305.79984964514
496 Train Loss 19915.074 Test Loss 6804.5993903614035
497 Train Loss 19920.406 Test Loss 6796.322016394876
498 Train Loss 19911.328 Test Loss 6795.773606748164
499 Train Loss 19914.107 Test Loss 6792.8367953355455
500 Train Loss 19910.062 Test Loss 6793.36854066

650 Train Loss 19867.262 Test Loss 7288.4152957757715
651 Train Loss 19586.592 Test Loss 7041.438571338865
652 Train Loss 19585.26 Test Loss 7042.107834932547
653 Train Loss 19987.59 Test Loss 7392.849276558104
654 Train Loss 19583.793 Test Loss 7042.633859113733
655 Train Loss 26144.477 Test Loss 13366.46519381341
656 Train Loss 19583.342 Test Loss 7042.948914320597
657 Train Loss 19603.46 Test Loss 7050.284214261935
658 Train Loss 19582.545 Test Loss 7042.5408684377
659 Train Loss 19580.783 Test Loss 7223.625214761427
660 Train Loss 19575.668 Test Loss 7125.931875757391
661 Train Loss 19579.135 Test Loss 6959.663925179015
662 Train Loss 19574.023 Test Loss 7058.477448996023
663 Train Loss 19572.941 Test Loss 7023.43702981903
664 Train Loss 19571.28 Test Loss 7076.494159663885
665 Train Loss 19570.45 Test Loss 7087.426462658098
666 Train Loss 19569.19 Test Loss 7125.003332695733
667 Train Loss 19568.244 Test Loss 7139.985055521406
668 Train Loss 19567.656 Test Loss 7159.781297134713
6

...

1288 Train Loss 18162.822 Test Loss 10363.486944002774
1289 Train Loss 18157.814 Test Loss 10309.517486821042
1290 Train Loss 18154.943 Test Loss 10365.050482013368
1291 Train Loss 18150.6 Test Loss 10465.958885117003
1292 Train Loss 18153.531 Test Loss 10595.992732566556
1293 Train Loss 18148.92 Test Loss 10519.771262553264
1294 Train Loss 18147.438 Test Loss 10645.563196505695
1295 Train Loss 18145.807 Test Loss 10709.595624627747
1296 Train Loss 18147.312 Test Loss 10832.749578145907
1297 Train Loss 18145.031 Test Loss 10748.631595689481
1298 Train Loss 18140.855 Test Loss 10704.577774994552
1299 Train Loss 18422.898 Test Loss 10081.341000552738
1300 Train Loss 18138.373 Test Loss 10666.008181962474
1301 Train Loss 18159.508 Test Loss 10480.796137200716
1302 Train Loss 18137.102 Test Loss 10627.362036726498
1303 Train Loss 18133.35 Test Loss 10623.613806389538
1304 Train Loss 18130.855 Test Loss 10634.970526860625
1305 Train Loss 18129.732 Test Loss 10593.038723216667
1306 Train Los

1449 Train Loss 17884.248 Test Loss 10749.901723361289
1450 Train Loss 17881.893 Test Loss 10724.961659420525
1451 Train Loss 17880.762 Test Loss 10674.667030331562
1452 Train Loss 17879.887 Test Loss 10713.667376603471
1453 Train Loss 17878.76 Test Loss 10768.476807060333
1454 Train Loss 17877.762 Test Loss 10778.607382969662
1455 Train Loss 17876.45 Test Loss 10776.869311192679
1456 Train Loss 17881.102 Test Loss 10949.094461405875
1457 Train Loss 17875.953 Test Loss 10820.440924546368
1458 Train Loss 17874.154 Test Loss 10805.963959946439
1459 Train Loss 17873.32 Test Loss 10796.52740837026
1460 Train Loss 17872.137 Test Loss 10782.610699173465
1461 Train Loss 17871.309 Test Loss 10786.33619694011
1462 Train Loss 17870.07 Test Loss 10789.67012046151
1463 Train Loss 17868.781 Test Loss 10747.995701595146
1464 Train Loss 17867.004 Test Loss 10709.557272115673
1465 Train Loss 18225.969 Test Loss 10914.386018227697
1466 Train Loss 17865.732 Test Loss 10671.636592540348
1467 Train Loss 2

1604 Train Loss 17363.33 Test Loss 17033.64190163908
1605 Train Loss 17359.438 Test Loss 17472.32816361751
1606 Train Loss 12392247.0 Test Loss 12697943.075406494
1607 Train Loss 1935229.2 Test Loss 1993941.024958485
1608 Train Loss 17359.477 Test Loss 17472.46190101535
1609 Train Loss 17359.479 Test Loss 17471.682318532872
1610 Train Loss 17359.438 Test Loss 17472.32814802578
1611 Train Loss 12392247.0 Test Loss 12697943.075406494
1612 Train Loss 1935229.2 Test Loss 1993941.024958485
1613 Train Loss 17359.477 Test Loss 17472.46190101535
1614 Train Loss 17359.479 Test Loss 17471.682318532872
1615 Train Loss 17359.438 Test Loss 17472.32814802578
1616 Train Loss 12392247.0 Test Loss 12697943.075406494
1617 Train Loss 1935229.2 Test Loss 1993941.024958485
1618 Train Loss 17359.477 Test Loss 17472.46190101535
1619 Train Loss 17359.479 Test Loss 17471.682318532872
1620 Train Loss 17359.438 Test Loss 17472.32814802578
1621 Train Loss 12392247.0 Test Loss 12697943.075406494
1622 Train Loss 19

1758 Train Loss 17359.477 Test Loss 17472.46190101535
1759 Train Loss 17359.479 Test Loss 17471.682318532872
1760 Train Loss 17359.438 Test Loss 17472.32814802578
1761 Train Loss 12392247.0 Test Loss 12697943.075406494
1762 Train Loss 1935229.2 Test Loss 1993941.024958485
1763 Train Loss 17359.477 Test Loss 17472.46190101535
1764 Train Loss 17359.479 Test Loss 17471.682318532872
1765 Train Loss 17359.438 Test Loss 17472.32814802578
1766 Train Loss 12392247.0 Test Loss 12697943.075406494
1767 Train Loss 1935229.2 Test Loss 1993941.024958485
1768 Train Loss 17359.477 Test Loss 17472.46190101535
1769 Train Loss 17359.479 Test Loss 17471.682318532872
1770 Train Loss 17359.438 Test Loss 17472.32814802578
1771 Train Loss 12392247.0 Test Loss 12697943.075406494
1772 Train Loss 1935229.2 Test Loss 1993941.024958485
1773 Train Loss 17359.477 Test Loss 17472.46190101535
1774 Train Loss 17359.479 Test Loss 17471.682318532872
1775 Train Loss 17359.438 Test Loss 17472.32814802578
1776 Train Loss 12

KeyboardInterrupt: 

In [ ]:
xt_test_tensor

In [14]:
flag = 1
for reps in range(max_reps):
    print(reps)
    N_u = 1000 #Total number of data points for 'u'
    N_f = 10000 #Total number of collocation points 
    
    X_f_train_np_array, X_u_train_np_array, u_train_np_array = trainingdata(N_u,N_f,reps*32)
        
    X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
    X_u_train = torch.from_numpy(X_u_train_np_array).float().to(device)
    u_train = torch.from_numpy(u_train_np_array).float().to(device)
        
    #u = torch.from_numpy(u_true).float().to(device)
    f_hat = torch.zeros(X_f_train.shape[0],1).to(device)
        

    #X_u_test_tensor = torch.from_numpy(X_u_test).float().to(device)
    'Convert to tensor and send to GPU'


#     layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

#     #layers = np.array([2,512,512,1])
#     PINN = Sequentialmodel(layers)
       
#     PINN.to(device)

#     'Neural Network Summary'
#     print(PINN)

#     params = list(PINN.parameters())


    'L-BFGS Optimizer'
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                                  max_iter = 10000, 
                                  max_eval = None, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')

    start_time = time.time()

    optimizer.step(PINN.closure)
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(PINN.train_loss)
    test_loss_full.append(PINN.test_loss)
    elapsed_time[reps] = time.time() - start_time
    beta_full.append(PINN.beta_val)
    

mdic = {"train_loss": train_loss_full,"test_loss": test_loss_full,"Time": elapsed_time, "beta": beta_full, "label": label}
savemat(label+'.mat', mdic) 

0
1894 Train Loss 5.2800687e+16 Test Loss 17471.682318532872
1895 Train Loss 4.2560614e+17 Test Loss 24085.92891815221
1896 Train Loss 2.380986e+18 Test Loss 21581.492381383745
1897 Train Loss 2.4356868e+17 Test Loss 18308.40552833872
1898 Train Loss 3.930569e+17 Test Loss 21136.233481193765
1899 Train Loss 7.225028e+17 Test Loss 19554.393852808756
1900 Train Loss 9.637539e+17 Test Loss 18846.313377289713
1901 Train Loss 2.5603874e+16 Test Loss 19525.330767421183
1902 Train Loss 3.8842486e+16 Test Loss 19978.867235658734
1903 Train Loss 3.3151521e+16 Test Loss 20250.490489461088
1904 Train Loss 3.0141492e+16 Test Loss 19982.166467520972
1905 Train Loss 3.0620173e+16 Test Loss 19783.414762038152
1906 Train Loss 2.6086946e+16 Test Loss 19662.94316035007
1907 Train Loss 2.5634512e+16 Test Loss 19596.112033297442
1908 Train Loss 2.5681549e+16 Test Loss 19560.786687886743
1909 Train Loss 2.564066e+16 Test Loss 19543.35821714864
1910 Train Loss 2.5620043e+16 Test Loss 19533.985557348107
1911

2033 Train Loss 17588175000000.0 Test Loss 267907.3505355942
2034 Train Loss 16674166000000.0 Test Loss 25214.76743052457
2035 Train Loss 16493186000000.0 Test Loss 28104.278706364385
2036 Train Loss 16308021000000.0 Test Loss 50089.03067686489
2037 Train Loss 16102822000000.0 Test Loss 110262.41174265438
2038 Train Loss 15936685000000.0 Test Loss 211588.83072117262
2039 Train Loss 15908504000000.0 Test Loss 263098.6826723895
2040 Train Loss 15706920000000.0 Test Loss 165269.532523306
2041 Train Loss 15919086000000.0 Test Loss 120991.00476972047
2042 Train Loss 15489145000000.0 Test Loss 41178.922335582974
2043 Train Loss 15109935000000.0 Test Loss 24910.303299770276
2044 Train Loss 14186638000000.0 Test Loss 75762.00011601584
2045 Train Loss 13278542000000.0 Test Loss 1615503.3483147519
2046 Train Loss 17553701000000.0 Test Loss 689139.575699256
2047 Train Loss 12956870000000.0 Test Loss 1500990.409872352
2048 Train Loss 12635795000000.0 Test Loss 1389588.695815312
2049 Train Loss 124

2169 Train Loss 2719508600000.0 Test Loss 258941.90679227264
2170 Train Loss 2710950400000.0 Test Loss 219718.96250994282
2171 Train Loss 2704188600000.0 Test Loss 269994.9854066796
2172 Train Loss 2698031000000.0 Test Loss 306298.30665541167
2173 Train Loss 2694738000000.0 Test Loss 347399.9585373647
2174 Train Loss 2692304000000.0 Test Loss 325229.5773105295
2175 Train Loss 2687244700000.0 Test Loss 303166.8782849654
2176 Train Loss 2679073100000.0 Test Loss 337275.84706452757
2177 Train Loss 2660469300000.0 Test Loss 392603.04761367274
2178 Train Loss 2644038600000.0 Test Loss 400872.28904201376
2179 Train Loss 2632029300000.0 Test Loss 348110.99760186224
2180 Train Loss 2616206600000.0 Test Loss 296098.5431591096
2181 Train Loss 2599668400000.0 Test Loss 258287.368912799
2182 Train Loss 2582604400000.0 Test Loss 232563.97147845716
2183 Train Loss 2642463400000.0 Test Loss 209145.42661023684
2184 Train Loss 2568137700000.0 Test Loss 222895.34560512574
2185 Train Loss 2557130300000.0

2308 Train Loss 956479100000.0 Test Loss 57383.35152875947
2309 Train Loss 951462200000.0 Test Loss 55958.18280928134
2310 Train Loss 1008178400000.0 Test Loss 48987.728857686576
2311 Train Loss 945305300000.0 Test Loss 52667.116988465066
2312 Train Loss 940803700000.0 Test Loss 53102.60007185975
2313 Train Loss 938620700000.0 Test Loss 49332.985244071664
2314 Train Loss 930675750000.0 Test Loss 49617.707680993226
2315 Train Loss 1349196400000.0 Test Loss 162642.169069399
2316 Train Loss 930052600000.0 Test Loss 48506.99415966997
2317 Train Loss 948751760000.0 Test Loss 46421.86883387837
2318 Train Loss 925451750000.0 Test Loss 47578.45664762904
2319 Train Loss 917691960000.0 Test Loss 51816.95501984857
2320 Train Loss 904341550000.0 Test Loss 61216.324123564475
2321 Train Loss 894555000000.0 Test Loss 63674.37714298044
2322 Train Loss 881980100000.0 Test Loss 61724.47927366048
2323 Train Loss 873474200000.0 Test Loss 57889.30674863803
2324 Train Loss 860841400000.0 Test Loss 49489.803

2447 Train Loss 372304840000.0 Test Loss 36039.381343531895
2448 Train Loss 370366500000.0 Test Loss 41440.06856552775
2449 Train Loss 359415150000.0 Test Loss 36123.76340449524
2450 Train Loss 360340780000.0 Test Loss 34451.10999541301
2451 Train Loss 354423370000.0 Test Loss 34252.47915483915
2452 Train Loss 352763940000.0 Test Loss 33900.229470490856
2453 Train Loss 349964800000.0 Test Loss 34094.55101690613
2454 Train Loss 347221600000.0 Test Loss 39391.1920295978
2455 Train Loss 344824000000.0 Test Loss 39715.55057788314
2456 Train Loss 342182850000.0 Test Loss 35092.971003309576
2457 Train Loss 341375160000.0 Test Loss 34127.952895450086
2458 Train Loss 339887720000.0 Test Loss 33592.450863366954
2459 Train Loss 338604820000.0 Test Loss 34545.59235280192
2460 Train Loss 337437620000.0 Test Loss 35873.53477900018
2461 Train Loss 335671260000.0 Test Loss 35662.05060827316
2462 Train Loss 334410680000.0 Test Loss 35529.98438667566
2463 Train Loss 333000740000.0 Test Loss 34709.79648

2585 Train Loss 188375330000.0 Test Loss 3103.2527154553222
2586 Train Loss 187322070000.0 Test Loss 3645.6440261404246
2587 Train Loss 204122910000.0 Test Loss 45759.88568091841
2588 Train Loss 186604930000.0 Test Loss 9080.438123277163
2589 Train Loss 185313130000.0 Test Loss 10472.467149859172
2590 Train Loss 183548640000.0 Test Loss 19469.628107784352
2591 Train Loss 183708000000.0 Test Loss 7737.4878519628455
2592 Train Loss 182841200000.0 Test Loss 13108.046576214932
2593 Train Loss 184824700000.0 Test Loss 4677.818037268993
2594 Train Loss 182248390000.0 Test Loss 9784.843095746339
2595 Train Loss 180549500000.0 Test Loss 9329.357439934456
2596 Train Loss 185401840000.0 Test Loss 4950.113441730663
2597 Train Loss 177962800000.0 Test Loss 7324.0612833175555
2598 Train Loss 196174480000.0 Test Loss 3303.4882143783902
2599 Train Loss 177591880000.0 Test Loss 5974.941775546758
2600 Train Loss 175257570000.0 Test Loss 6280.6041704474555
2601 Train Loss 173316670000.0 Test Loss 11229.

2727 Train Loss 86328090000.0 Test Loss 5497.60393671112
2728 Train Loss 86162220000.0 Test Loss 8188.376201468327
2729 Train Loss 86121130000.0 Test Loss 8583.959416820822
2730 Train Loss 85984560000.0 Test Loss 8202.635388037346
2731 Train Loss 85641280000.0 Test Loss 8416.92119176071
2732 Train Loss 85377750000.0 Test Loss 9535.255653483642
2733 Train Loss 85855390000.0 Test Loss 10426.838946783777
2734 Train Loss 85185225000.0 Test Loss 9822.426486227485
2735 Train Loss 84960264000.0 Test Loss 9991.099374468617
2736 Train Loss 84771360000.0 Test Loss 9015.157439883147
2737 Train Loss 84677665000.0 Test Loss 8507.779776359004
2738 Train Loss 84637565000.0 Test Loss 7574.03431940529
2739 Train Loss 84569450000.0 Test Loss 6777.440364662725
2740 Train Loss 84489080000.0 Test Loss 5468.801852420853
2741 Train Loss 84459070000.0 Test Loss 5499.4379947846555
2742 Train Loss 86100710000.0 Test Loss 5371.069868690538
2743 Train Loss 84363354000.0 Test Loss 5472.217503180575
2744 Train Loss

2873 Train Loss 83653090000.0 Test Loss 6013.340110020088
2874 Train Loss 83666890000.0 Test Loss 6456.638862528729
2875 Train Loss 83654570000.0 Test Loss 6610.732380779041
2876 Train Loss 83637960000.0 Test Loss 6623.351591104441
2877 Train Loss 83632870000.0 Test Loss 6624.055917899242
2878 Train Loss 83653090000.0 Test Loss 6013.340110020088
2879 Train Loss 83666890000.0 Test Loss 6456.638862528729
2880 Train Loss 83654570000.0 Test Loss 6610.732380779041
2881 Train Loss 83637960000.0 Test Loss 6623.351591104441
2882 Train Loss 83632870000.0 Test Loss 6624.055917899242
2883 Train Loss 83653090000.0 Test Loss 6013.340110020088
2884 Train Loss 83666890000.0 Test Loss 6456.638862528729
2885 Train Loss 83654570000.0 Test Loss 6610.732380779041
2886 Train Loss 83637960000.0 Test Loss 6623.351591104441
2887 Train Loss 83632870000.0 Test Loss 6624.055917899242
2888 Train Loss 83653090000.0 Test Loss 6013.340110020088
2889 Train Loss 83666890000.0 Test Loss 6456.638862528729
2890 Train Los

KeyboardInterrupt: 

In [ ]:
PINN.W2

In [ ]:
u_pred = PINN.test(xt_test_tensor)

cmap = plt.cm.rainbow
plt.imshow(u_pred.reshape(100,256),cmap = cmap,aspect =1,vmin=-10,vmax=10)

In [ ]:
X_u_train

In [ ]:
np.min(u_pred)

In [ ]:
np.max(u_pred)

In [ ]:
cmap = plt.cm.rainbow
plt.imshow(u_true.reshape(100,256),cmap = cmap,aspect = 1,vmin=-10,vmax=10)

In [ ]:
np.min(u_true)

In [ ]:
cmap = plt.cm.rainbow
plt.imshow(np.transpose(np.abs(u_pred - u_true).reshape(100,256)),cmap = cmap,aspect = 0.2)

In [ ]:
a = 0 
for i in range(10):
    print(test_loss_full[i][-1])
    a = a + test_loss_full[i][-1]
print(a/10)